In [6]:
from pathlib import Path

from pandas import DataFrame

from src import load_base_model, results_path

all_attributes = ["model", "run", "signal", "load", "case", "LIA", "continents"]


def process_files(
    path: Path = results_path,
):
    data = {}
    # Loops on files.
    for model_path in path.glob("*"):
        for run_path in model_path.joinpath("runs").glob("*"):
            for file_path in run_path.joinpath("load").glob("*/*/*/*/*/barystatic_rise_mean_trend.json"):
                barystatic_sea_level = load_base_model(path=file_path.parent, name=file_path.name)
                parents = str(file_path.parent).split("/")
                # Line.
                run_options = run_path.name.split("__")
                model_parts = model_path.name.split("_____")
                model = "_____".join(
                    model_parts
                    if not "" in run_options
                    else (model_parts[:2] if not "short_term_anelasticity" in run_options else (model_parts[0], model_parts[2]))
                )
                line = {
                    "model": model,
                    "run": run_path.name.split("bounded")[0],
                    "signal": parents[-5],
                    "load": parents[-4],
                    "case": parents[-3],
                    "LIA": parents[-2],
                    "continents": parents[-1],
                    "barystatic_difference_with_elastic": round(
                        number=barystatic_sea_level["anelastic"] - barystatic_sea_level["elastic"], ndigits=5
                    ),
                }
                for key, value in line.items():
                    if not key in data.keys():
                        data[key] = []
                    data[key] += [value]
    return DataFrame(data).drop_duplicates()


# Example usage:
df = process_files()
df.to_csv(results_path.joinpath("barystatic_sea_level_differences_with_elastic.csv"))

In [7]:
from pandas import read_csv

df = read_csv(results_path.joinpath("barystatic_sea_level_differences_with_elastic.csv"))
df

,Unnamed: 0,model,run,signal,load,case,LIA,continents,barystatic_difference_with_elastic
0,0,PREM_____Benjamin_Q_QM1____asymptotic_mu_ratio...,__short_term_anelasticity__,ocean_load,Frederikse,upper,without_LIA,with_opposite_load_on_continents,0.18317
1,1,PREM_____Benjamin_Q_QM1____asymptotic_mu_ratio...,__short_term_anelasticity__,ocean_load,Frederikse,upper,without_LIA,without_opposite_load_on_continents,0.16587
2,2,PREM_____Benjamin_Q_QM1____asymptotic_mu_ratio...,__short_term_anelasticity__,ocean_load,Frederikse,upper,with_LIA,with_opposite_load_on_continents,0.17914
3,3,PREM_____Benjamin_Q_QM1____asymptotic_mu_ratio...,__short_term_anelasticity__,ocean_load,Frederikse,upper,with_LIA,without_opposite_load_on_continents,0.16297
4,4,PREM_____Benjamin_Q_QM1____asymptotic_mu_ratio...,__short_term_anelasticity__,ocean_load,Frederikse,mean,without_LIA,with_opposite_load_on_continents,0.19581
...,...,...,...,...,...,...,...,...,...
2095,5395,PREM_____Lau_2016____eta_m__ASTHENOSPHERE__3e+...,long_term_anelasticity__short_term_anelasticity__,ocean_load,Frederikse,mean,with_LIA,without_opposite_load_on_continents,0.16495
2096,5396,PREM_____Lau_2016____eta_m__ASTHENOSPHERE__3e+...,long_term_anelasticity__short_term_anelasticity__,ocean_load,Frederikse,lower,without_LIA,with_opposite_load_on_continents,0.20999
2097,5397,PREM_____Lau_2016____eta_m__ASTHENOSPHERE__3e+...,long_term_anelasticity__short_term_anelasticity__,ocean_load,Frederikse,lower,without_LIA,without_opposite_load_on_continents,0.18542
2098,5398,PREM_____Lau_2016____eta_m__ASTHENOSPHERE__3e+...,long_term_anelasticity__short_term_anelasticity__,ocean_load,Frederikse,lower,with_LIA,with_opposite_load_on_continents,0.19882


In [8]:
df.groupby(["run"]).agg({"barystatic_difference_with_elastic": ["mean", "min", "max", "std"]})

barystatic_difference_with_elastic  \
                                                                                mean   
run                                                                                    
__short_term_anelasticity__                                                 0.182056   
long_term_anelasticity____                                                  0.031217   
long_term_anelasticity__short_term_anelasticity__                           0.211891   

                                                                               
                                                       min      max       std  
run                                                                            
__short_term_anelasticity__                        0.13717  0.24571  0.022760  
long_term_anelasticity____                         0.00657  0.06343  0.015684  
long_term_anelasticity__short_term_anelasticity__  0.14320  0.30047  0.028046

In [4]:
# LIA variability.
df.groupby(["model", "run", "signal", "load", "case", "continents"]).agg(
    {"barystatic_difference_with_elastic": ["std"]}
).groups()

barystatic_difference_with_elastic  \
                                                                                                                                                                                                               mean   
model                                              run                                               signal     load       case  LIA         continents                                                               
PREM-----Benjamin_Q_PAR3P____asymptotic_mu_rati... __short_term_anelasticity__                       ocean_load Frederikse lower with_LIA    with_opposite_load_on_continents                               0.22573   
                                                                                                                                             without_opposite_load_on_continents                            0.21616   
                                                                                                                                 without_LIA with_opposite_load_on_continents                               0.23082   
                                                                                                                                             without_opposite_load_on_continents                            0.21981   
                                                                                                                           mean  with_LIA    with_opposite_load_on_continents                               0.20932   
...                                                                                                                                                                                                             ...   
PREM-----VM7____eta_m__ASTHENOSPHERE__4.5e+20--... long_term_anelasticity__short_term_anelasticity__ ocean_load Frederikse mean  without_LIA without_opposite_load_on_continents                            0.16972   
                                                                                                                           upper with_LIA    with_opposite_load_on_continents                               0.17122   
                                                                                                                                             without_opposite_load_on_continents                            0.15252   
                                                                                                                                 without_LIA with_opposite_load_on_continents                               0.18017   
                                                                                                                                             without_opposite_load_on_continents                            0.15937   

                                                                                                                                                                                           \
                                                                                                                                                                                      min   
model                                              run                                               signal     load       case  LIA         continents                                     
PREM-----Benjamin_Q_PAR3P____asymptotic_mu_rati... __short_term_anelasticity__                       ocean_load Frederikse lower with_LIA    with_opposite_load_on_continents     0.22573   
                                                                                                                                             without_opposite_load_on_continents  0.21616   
                                                                                                                                 without_LIA with_opposite_load_on_continents     0.23082   
                                